In [154]:
import pandas as pd
from docx import Document, types
import docx
import unicodedata
import re
import os
import glob
# doc = Document("/Users/kong/Documents/เอกสารประกาศ KB/25.ตรวจคัดกรองมะเร็งปากมดลูก/ตรวจคัดกรองมะเร็งปากมดลูก.docx")
def read_docx_table(document, table_num=1, nheader=1):
    table = document.tables[table_num-1]
    data = [[re.sub(r'\s+',' ',unicodedata.normalize("NFKD",cell.text).replace('\n','').replace('\t','').strip()) for cell in row.cells] for row in table.rows]
    df = pd.DataFrame(data)
    if nheader == 1:
        df = df.rename(columns=df.iloc[0]).drop(df.index[0]).reset_index(drop=True)
    elif nheader == 2:
        outside_col, inside_col = df.iloc[0], df.iloc[1]
        hier_index = pd.MultiIndex.from_tuples(list(zip(outside_col, inside_col)))
        df = pd.DataFrame(data, columns=hier_index).drop(df.index[[0,1]]).reset_index(drop=True)
    elif nheader > 2:
        raise Exception("More than two headers not currently supported")
    return df

def read_p_tb(path):
    doc = Document(path)
    tb_count = 1
    passages = ['']
    passage = ''
    for idx , element in enumerate(doc.element.body):
        id = len(passages)
        if id == 0:
            id = 0
        else:
            id -= 1
        # print(element.tag)
        if element.tag.endswith('p'):  # Check if the element is a paragraph
            paragraph = element
            line = paragraph.text
            line = unicodedata.normalize("NFKD",line).replace('\n','').replace('\t','').strip()
            line = re.sub(r'\s+',' ',line)
            if line not in ['',' ']:
                if len(passage)+len(line)<8001:
                    passage+=' '+line
                    passages[id] = passage
                else:
                    passage = line
                    passages.append(passage)
                # print(line)  # Print or process the paragraph text
        elif element.tag.endswith('tbl'):  # Check if the element is a table
            tb = read_docx_table(doc, table_num=tb_count, nheader=1)
            tb_dict = f"ตารางมีคอลัมน์ {list(tb.columns)}: {tb.to_dict('records')}"
            if len(passage)+len(tb_dict)<8001:
                passage+=' '+tb_dict
                passages[id] = passage
            else:
                passage = tb_dict
                passages.append(passage)
            # print(tb_dict)
            tb_count+=1

        elif element.tag.endswith('sectPr'):
            pass

    return passages

def upadate_new_docx(path,data_path):
    data = pd.read_csv(data_path)
    passages = read_p_tb(path)
    file_name = [os.path.basename(path).split('.')[0]] * len(passages)
    file_path = [path]* len(passages)
    for i in range(len(passages)):
        new_data = {'File_name':file_name[i],'File_path':file_path[i],'Passage':passages[i]}
        data = data._append(new_data, ignore_index = True)
    # print(data['Passage'])
    data.to_csv(data_path,index=False)
    print('update success')

# read_docx_table(doc).to_dict('records')

In [11]:
# path = "/Users/kong/Documents/mod/RAG_CC67/folder_rag/ตรวจคัดกรองมะเร็งปากมดลูก.docx"
# passage = read_p_tb(path)
# passage

[' ตรวจคัดกรองมะเร็งปากมดลูก กลุ่มเป้าหมาย : หญิงที่มีสัญชาติไทย มีเลขบัตรประจําตัวประชาชน 13 หลัก ทุกสิทธิการรักษา รวมถึงสิทธิว่าง (ยกเว้นต่างด้าว) 1. อายุ 30 - 59 ปี ทุกคน 2. อายุ 15-29 ปีที่มีความเสี่ยงสูง ทั้งนี้ขึ้นอยู่ดุลยพินิจของแพทย์สั่งตรวจคัดกรอง เช่น มีคู่นอนหลายคน มีเพศสัมพันธ์ตั้งแต่อายุยังน้อย หรือมีเพศสัมพันธ์โดยไม่ใส่ถุงยางอนามัย หมายเหตุ : สปสช.ประชาสัมพันธ์ผ่าน Facebook live เชิญชวน หากมีผู้ประกอบการ บริษัท โรงงาน หน่วยงาน องค์กรต่างๆ ฯลฯ ติดต่อมา 1330 ต้องการให้หน่วยบริการลงพื้นที่เพื่อตรวจคัดกรองมะเร็งปากลูกหรือตรวจด้วยตนเองด้วยวิธี HPV Self test ให้กับพนักงานหญิงจํานวนมาก ให้1330 รับเรื่องเข้ามาก่อน เปิดประเด็นบริการ--->การตรวจคัดกรองมะเร็งปากมดลูก ---โอนทีมคุ้มครองสิทธิ ทางทีมคุ้มครองสิทธิจะประสานสปสช.เขตในพื้นที่เพื่อจัดหาหน่วยบริการลงพื้นที่ตรวจให้ //กรณีพื้นที่กทม. เงื่อนไขคือ จะต้องมีพนักงานหญิง อายุ 30-59 ปี จํานวน 30 คนขึ้นไป ให้1330 รับเรื่องเข้ามาและทีมคุ้มครองสิทธิ ประสานคุณรุ่งระวี เจ้าหน้าที่สปสช. กทม ได้เลย (อัปเดต 15/1/67 เวลา 14.00 น.) ข่าวเริ่มแคมเป

In [274]:
# passage[0]

" บริการฟื้นฟูสมรรถภาพด้านการแพทย์ 1. คนพิการได้รับอุปกรณ์เครื่องช่วยคนพิการตามความจําเป็นอย่างทั่วถึงและครอบคลุม 2. คนพิการ ผู้สูงอายุที่จําเป็นต้องได้รับการฟื้นฟูสมรรถภาพด้านการแพทย์และผู้ป่วยที่จําเป็นต้องได้รับการฟื้นฟูสมรรถภาพด้านการแพทย์ ได้รับบริการด้านการฟื้นฟูสมรรถภาพด้านการแพทย์อย่างต่อเนื่องทั้งในหน่วยบริการและในชุมชน 3. ส่งเสริมให้เกิดความร่วมมือระหว่างหน่วยบริการ องค์กรปกครองส่วนท้องถิ่น องค์กรคนพิการองค์กรภาคีที่เกี่ยวข้อง และชุ มชน ในการพัฒนารูปแบบการดูแล ช่วยเหลือ สนับสนุน และให้บริการฟื้นฟูสมรรถภาพแก่กลุ่มเป้าหมายร่วมกัน ทั้งในหน่วยบริการและในชุมชนอย่างยั่งยืน 1. กลุ่มเป้าหมาย ครอบคลุมผู้มีสิทธิหลักประกันสุขภาพแห่งชาติ 3 กลุ่ม ดังนี้ 1. คนพิการ (รหัสสิทธิย่อย 74) 2. ผู้สูงอายุที่จําเป็นต้องได้รับการฟื้นฟูสมรรถภาพด้านการแพทย์ 3. ผู้ป่วยที่จําเป็นต้องได้รับการฟื้นฟูสมรรถภาพด้านการแพทย์ 2. สิทธิประโยชน์ที่ได้รับ ครอบคลุมบริการ ค่าอุปกรณ์เครื่องช่วยสําหรับคนพิการ ค่าบริการฟื้นฟูสมรรถภาพด้านการแพทย์ระยะกลาง (Intermediate care :IMC) การฝึกทักษะการดํารงชีวิตอิสระสําหรับคนพิกา

In [156]:
data_path = '/Users/kong/Documents/mod/RAG_CC67/folder_rag/Data.csv'
# path = "file/folder_rag/"
# for i in glob.glob(path + '*.docx'):
#     # print(i)
#     upadate_new_docx(i,data_path)

In [157]:
data = pd.read_csv(data_path)
data

,File_name,File_path,Passage
0,ตรวจคัดกรองมะเร็งปากมดลูก,file/folder_rag/ตรวจคัดกรองมะเร็งปากมดลูก.docx,ตรวจคัดกรองมะเร็งปากมดลูก กลุ่มเป้าหมาย : หญิ...
1,การใช้สิทธิย่อยคนพิการ,file/folder_rag/การใช้สิทธิย่อยคนพิการ.docx,การใช้สิทธิย่อยคนพิการ (ท74) 1.กลุ่มเป้าหมาย ...
2,สิทธิไม่ตรงตามจริง ติดสิทธิประกันสังคม,file/folder_rag/สิทธิไม่ตรงตามจริง ติดสิทธิประ...,ติดสิทธิประกันสังคม (สิทธิไม่ตรงตามจริง) กลุ่...
3,บริการฟื้นฟูสมรรถภาพด้านการแพทย์,file/folder_rag/บริการฟื้นฟูสมรรถภาพด้านการแพท...,บริการฟื้นฟูสมรรถภาพด้านการแพทย์ 1. คนพิการได...
4,การใช้สิทธิว่าง,file/folder_rag/การใช้สิทธิว่าง.docx,การใช้สิทธิว่าง (มาตรา8 ตามพ.ร.บ.หลักประกันสุ...


In [158]:
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
import tiktoken
# from tqdm import tqdm
from tqdm.auto import tqdm  # for notebooks
import string
# from FlagEmbedding import BGEM3FlagModel
import numpy as np
tqdm.pandas()
os.environ["OPENTYPHOON_API_KEY"] = "sk-PLQ1EWaCPQZc0guiwHdCQMYUe3GsTZGFEdg3uLI2qcRKz4mA"
client = OpenAI(api_key=os.environ["OPENTYPHOON_API_KEY"], base_url="https://api.opentyphoon.ai/v1")

In [159]:
yn_prompt = '''
You are an Assistant responsible for helping detect whether the retrieved document is relevant to the query. For a given input, you need to output a single token: "Yes" or "No" indicating the retrieved document is relevant to the query.

Query: {query}
Document: """{document}"""
Relevant:
'''

In [160]:
list_document ='''TITLE: {title} 
DOCUMENT: """"{document}"""'''

ans_prompt ='''
You are a helpful assistant that must try your best effort to answer the user question ALWAYS following this guidelines:

Keep your answer ground in the facts provided in DOCUMENT section.
If the DOCUMENT section doesn’t contain the facts to answer the QUESTION ALWAYS return [NONE].

{template}

QUESTION: {question}
ANSWER:
'''


INSTRUCTIONS
Read the QUESTION carefully.
Examine the DOCUMENT section for any passages that provide the facts required to answer the QUESTION.
If the DOCUMENT contains the necessary information, extract and return the relevant passage.
If the DOCUMENT does not contain the necessary information, return [NONE].
Your extracted passage or [NONE] must directly address the QUESTION.

In [161]:
extract_prompt = """
bring one or more sentences which relate with QUESTION from DOCUMENT
QUESTION
{query}

DOCUMENT
{document}

OUTPUT

"""

In [162]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def retrival_70b(query,document):
    response = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[{"role": "user", "content": yn_prompt.format(query=query, document=document)}],
        max_tokens=1,
        temperature= 0,
        top_p = 0.9,
        seed=55
    )

    return response.choices[0].message.content

# @retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def extract_passage(query,document):
    response = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[{"role": "user", "content": extract_prompt.format(query=query, document=document)}],
        # max_tokens=400,
        temperature= 0.3,
        top_p = 0.9,
        seed=55
    )

    return response.choices[0].message.content

def clean_yn(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = text.translate(str.maketrans('', '', string.digits))
    return text.strip()

def answer(question,template):
    response = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[{"role": "user", "content": ans_prompt.format(question=question, template=template)}],
        max_tokens=4096,
        temperature= 0.3,
        top_p = 0.9,
        seed=55
    )

    return response.choices[0].message.content

def chat_rag(query,docs):
    global temp
    f_yn = []
    for i in docs['Passage']:
        f_yn.append(clean_yn(retrival_70b(query,i)))
    if 'yes' in f_yn:
        idx = [i for i,x in enumerate(f_yn) if x=='yes']
    else:
        return 'None'
    print(idx)
    contents = []
    for j,content in zip(idx,docs['Passage'][idx]):
        print(f'{[j]}')
        print(content)
        Ans = extract_passage(query,content)
        contents.append(Ans)
        print(Ans)
    template = '\n'.join([list_document.format(title=i, document = j) for i,j in zip(docs['File_name'][idx],contents)])
    ans = answer(query,template)
    if clean_yn(ans) == 'none':
        return ans
    else: 
        return (ans,idx)
    

In [164]:
tb = chat_rag('หมาบินได้ไหม',data)
tb

'None'

In [146]:
# def transform_list(input_list):
#     result = []
#     temp = []
    
#     for item in input_list:
#         if isinstance(item, list):
#             if temp:
#                 result.append(temp)
#                 temp = []
#             result.append(item)
#         else:
#             temp.append(item)
    
#     if temp:
#         result.append(temp)
    
#     return result

# def merge_strings(strings, max_length=2000):
#     merged_strings = []
#     current_string = ""
    
#     for s in strings:
#         if len(current_string) + len(s) <= max_length:
#             current_string += s
#         else:
#             merged_strings.append(current_string)
#             current_string = s
            
#     if current_string:
#         merged_strings.append(current_string)
#     return merged_strings

# def read_docx(path):
#     doc = Document(path)
#     tb_count = 1
#     passages = []
#     idx_tb = []
#     id = -1
#     for idx , element in enumerate(doc.element.body):
#         if element.tag.endswith('p'):  # Check if the element is a paragraph
#             paragraph = element
#             line = paragraph.text
#             line = unicodedata.normalize("NFKD",line).replace('\n','').replace('\t','').strip()
#             line = re.sub(r'\s+',' ',line)
#             if line not in ['',' ']:
#                 passages.append(line)
#                 id+=1
#                 # print(line)  # Print or process the paragraph text
#         elif element.tag.endswith('tbl'):  # Check if the element is a table
#             tb = read_docx_table(doc, table_num=tb_count, nheader=1)
#             tb_template = f"ตารางมีคอลัมน์ {list(tb.columns)}: {tb.to_dict('records')}"
#             passages.append([tb_template])
#             id+=1
#             idx_tb.append(id)
#             tb_count+=1

#         elif element.tag.endswith('sectPr'):
#             pass
#     for i in passages:
#         print(i)
#     passages = transform_list(passages)
#     for i in range(len(passages)):
#         if i not in idx_tb:
#             passages[i] = merge_strings(passages[i])

#     [for i in passages]
#     return [item for sublist in passages for item in sublist]

In [147]:
# path = "/Users/kong/Documents/mod/RAG_CC67/folder_rag/ตรวจคัดกรองมะเร็งปากมดลูก.docx"
# p = read_docx(path)


ตรวจคัดกรองมะเร็งปากมดลูก
กลุ่มเป้าหมาย : หญิงที่มีสัญชาติไทย มีเลขบัตรประจําตัวประชาชน 13 หลัก ทุกสิทธิการรักษา รวมถึงสิทธิว่าง (ยกเว้นต่างด้าว)
1. อายุ 30 - 59 ปี ทุกคน
2. อายุ 15-29 ปีที่มีความเสี่ยงสูง ทั้งนี้ขึ้นอยู่ดุลยพินิจของแพทย์สั่งตรวจคัดกรอง เช่น มีคู่นอนหลายคน มีเพศสัมพันธ์ตั้งแต่อายุยังน้อย หรือมีเพศสัมพันธ์โดยไม่ใส่ถุงยางอนามัย
หมายเหตุ : สปสช.ประชาสัมพันธ์ผ่าน Facebook live เชิญชวน หากมีผู้ประกอบการ บริษัท โรงงาน หน่วยงาน องค์กรต่างๆ ฯลฯ ติดต่อมา 1330 ต้องการให้หน่วยบริการลงพื้นที่เพื่อตรวจคัดกรองมะเร็งปากลูกหรือตรวจด้วยตนเองด้วยวิธี HPV Self test ให้กับพนักงานหญิงจํานวนมาก ให้1330 รับเรื่องเข้ามาก่อน
เปิดประเด็นบริการ--->การตรวจคัดกรองมะเร็งปากมดลูก ---โอนทีมคุ้มครองสิทธิ ทางทีมคุ้มครองสิทธิจะประสานสปสช.เขตในพื้นที่เพื่อจัดหาหน่วยบริการลงพื้นที่ตรวจให้
//กรณีพื้นที่กทม. เงื่อนไขคือ จะต้องมีพนักงานหญิง อายุ 30-59 ปี จํานวน 30 คนขึ้นไป ให้1330 รับเรื่องเข้ามาและทีมคุ้มครองสิทธิ ประสานคุณรุ่งระวี เจ้าหน้าที่สปสช. กทม ได้เลย (อัปเดต 15/1/67 เวลา 14.00 น.)
ข่าวเริ่มแคมเปญ”ต

In [135]:
# p

['ตรวจคัดกรองมะเร็งปากมดลูกกลุ่มเป้าหมาย : หญิงที่มีสัญชาติไทย มีเลขบัตรประจําตัวประชาชน 13 หลัก ทุกสิทธิการรักษา รวมถึงสิทธิว่าง (ยกเว้นต่างด้าว)1. อายุ 30 - 59 ปี ทุกคน2. อายุ 15-29 ปีที่มีความเสี่ยงสูง ทั้งนี้ขึ้นอยู่ดุลยพินิจของแพทย์สั่งตรวจคัดกรอง เช่น มีคู่นอนหลายคน มีเพศสัมพันธ์ตั้งแต่อายุยังน้อย หรือมีเพศสัมพันธ์โดยไม่ใส่ถุงยางอนามัยหมายเหตุ : สปสช.ประชาสัมพันธ์ผ่าน Facebook live เชิญชวน หากมีผู้ประกอบการ บริษัท โรงงาน หน่วยงาน องค์กรต่างๆ ฯลฯ ติดต่อมา 1330 ต้องการให้หน่วยบริการลงพื้นที่เพื่อตรวจคัดกรองมะเร็งปากลูกหรือตรวจด้วยตนเองด้วยวิธี HPV Self test ให้กับพนักงานหญิงจํานวนมาก ให้1330 รับเรื่องเข้ามาก่อนเปิดประเด็นบริการ--->การตรวจคัดกรองมะเร็งปากมดลูก ---โอนทีมคุ้มครองสิทธิ ทางทีมคุ้มครองสิทธิจะประสานสปสช.เขตในพื้นที่เพื่อจัดหาหน่วยบริการลงพื้นที่ตรวจให้//กรณีพื้นที่กทม. เงื่อนไขคือ จะต้องมีพนักงานหญิง อายุ 30-59 ปี จํานวน 30 คนขึ้นไป ให้1330 รับเรื่องเข้ามาและทีมคุ้มครองสิทธิ ประสานคุณรุ่งระวี เจ้าหน้าที่สปสช. กทม ได้เลย (อัปเดต 15/1/67 เวลา 14.00 น.)ข่าวเริ่มแคมเปญ”ตรวจมะ

In [148]:
# re_prompt = '''
# """{document}"""
# Rewrite the text into paragraphs without using any pronouns in Thai. Make sure to maintain the original meaning and flow and each paragraph is not relate with other .
# '''

In [149]:
# def rewrite(document):
#     response = client.chat.completions.create(
#         model="typhoon-v1.5x-70b-instruct",
#         messages=[{"role": "user", "content": re_prompt.format(document=document)}],
#         max_tokens=2000,
#         temperature= 0.3,
#         top_p = 0.9,
#         seed=55
#     )

#     return response.choices[0].message.content

In [150]:
# print(rewrite(p[2]))

ช่องทางการตรวจสอบสิทธิประโยชน์บริการสร้างเสริมสุขภาพ PP (พีพี แพลตฟอร์ม) ด้วยตนเอง มี 3 ช่องทาง คือ ไลน์ OA สปสช. @nhso, แอปพลิเคชั่น สปสช., และหน้าเว็บไซต์หลัก สปสช.

ขั้นตอนการเข้าใช้งานแต่ละช่องทางมีขั้นตอน ดังนี้ สำหรับช่องทาง ไลน์ OA สปสช. @nhso ให้เลือกเมนู "เช็คสิทธิ ตรวจสุขภาพฟรี". สำหรับช่องทาง แอปพลิเคชั่น สปสช. ให้เลือกเมนู "เช็คสิทธิบริการ สร้างเสริมสุขภาพ". สำหรับช่องทาง หน้าเว็บไซต์หลัก สปสช. www.nhso.go.th ให้เลือกเมนู "เช็คสิทธิบริการสร้างเสริมสุขภาพ บริการสร้างเสริมสุขภาพและป้องกันโรค" และเข้าใช้งานผ่านลิงก์ https://ppplatform.nhso.go.th/web/form/login.

เมื่อเข้าใช้งานผ่านช่องทางใดช่องทางหนึ่ง จะต้องกรอกเลขบัตรประชาชน วัน/เดือน/ปีเกิด แล้วกดบันทึกข้อมูล ระบบจะแสดงหัวข้อ "สร้างเสริมสุขภาพและป้องกันโรค" ซึ่งประชาชนสามารถคลิกเข้าไปตรวจสอบได้ว่า รับบริการใดได้บ้าง เมื่อเลือกบริการเรียบร้อยแล้ว ระบบจะแสดงหน่วยบริการใกล้เคียงให้ทราบว่ามีหน่วยบริการใดบ้าง.

ช่องทางดังกล่าวจะดูบริการ PP ได้เท่านั้น แต่ไม่สามารถจองรับบริการได้ หากต้องการจอง แนะนำจองผ่าน App เป๋าตัง หรือเข้ารับ

In [151]:
# print(rewrite(p[0]))


กลุ่มเป้าหมายในการตรวจคัดกรองมะเร็งปากมดลูก คือ หญิงที่มีสัญชาติไทยและมีเลขบัตรประจำตัวประชาชน 13 หลัก ทุกสิทธิการรักษา รวมถึงสิทธิว่าง (ยกเว้นต่างด้าว) โดยแบ่งเป็น 2 กลุ่ม คือ อายุ 30 - 59 ปี ทุกคน และอายุ 15-29 ปีที่มีความเสี่ยงสูง ซึ่งขึ้นอยู่ดุลยพินิจของแพทย์สั่งตรวจคัดกรอง เช่น มีคู่นอนหลายคน มีเพศสัมพันธ์ตั้งแต่อายุยังน้อย หรือมีเพศสัมพันธ์โดยไม่ใส่ถุงยางอนามัย

สปสช.ประชาสัมพันธ์ผ่าน Facebook live เชิญชวน หากมีผู้ประกอบการ บริษัท โรงงาน หน่วยงาน องค์กรต่างๆ ฯลฯ ติดต่อมา 1330 เพื่อให้หน่วยบริการลงพื้นที่เพื่อตรวจคัดกรองมะเร็งปากลูกหรือตรวจด้วยตนเองด้วยวิธี HPV Self test ให้กับพนักงานหญิงจำนวนมาก

การตรวจคัดกรองมะเร็งปากมดลูกจะมีการโอนทีมคุ้มครองสิทธิ ทางทีมคุ้มครองสิทธิจะประสานสปสช.เขตในพื้นที่เพื่อจัดหาหน่วยบริการลงพื้นที่ตรวจให้ สำหรับพื้นที่กทม. เงื่อนไขคือ จะต้องมีพนักงานหญิง อายุ 30-59 ปี จำนวน 30 คนขึ้นไป ให้1330 รับเรื่องเข้ามาและทีมคุ้มครองสิทธิ ประสานคุณรุ่งระวี เจ้าหน้าที่สปสช. กทม ได้เลย

วิธีการใช้สิทธิ คือ แสดงบัตรประจำตัวประชาชน / เด็กอายุต่ำกว่า 15 ปี ใช้สูติบัตรหร